In [1]:
from sympy.combinatorics import Permutation
import numpy as np
import scipy.sparse as scpy

In [6]:
def perform_action(state, action):
    """
    perform the given action on the given state in-place
    for twisty puzzles this means applying the permutations of a move

    inputs:
    -------
        state - (list) of (int) - list representing the state
        action - (list) of (list) of (int) - list representing an action, here as a list of cycles
            cycles are lists of list indices of the state list.
    """
    for cycle in action: # loop over all cycles in the move
        j = cycle[0]
        for i in cycle:  # apply cycle
            state[i], state[j] = state[j], state[i]
    return state

In [28]:
def np_perm(state: np.ndarray, action: list[np.ndarray]):
    """
    """
    for cycle in action:
        state[cycle[0]] = state[cycle[-1]]
        state[cycle[1:]] = state[cycle[:-1]]
    return state

def list_to_arrays(state, action):
    """
    """
    state = np.array(state, dtype=np.int16)
    action = [np.array(cycle, dtype=np.int16) for cycle in action]
    return state, action

In [24]:
def non_cycle_np_perm(state: np.ndarray, action: list[np.ndarray]):
    # Create an array that maps the original indices to their new positions
    mapping = np.arange(state.size)
    
    # Apply the permutation for each cycle
    for cycle in action:
        mapping[cycle] = np.roll(cycle, -1)
    return mapping

In [32]:
def non_id_cycle_np_perm(state: np.ndarray, action: list[np.ndarray]):
    # Create an array that maps the original indices to their new positions
    mapping = np.arange(state.size)
    
    # Apply the permutation for each cycle
    for cycle in action:
        if cycle.size > 1:
            mapping[cycle] = np.roll(cycle, -1)
    non_identity_indices = np.where(mapping != np.arange(state.size))[0]
    return mapping[non_identity_indices]

In [33]:
def get_perm_matrix(perm, n):
    """
    convert a permutation in cyclic notation into a permutation matrix, such that M@s = perm(s)
    inputs:
        perm - (list) of (tuple) of (int) - permutation in cyclic notation
        n - (int) - number of elements in permutation
    returns:
        (np.array) - permutation matrix
    """
    M = np.eye(n, dtype=np.bool_)
    # M = np.eye(n, dtype=np.uint8)
    for cycle in perm: # loop over all cycles in the move
        j = cycle[0]
        # M[j, i] 
        for i in cycle:  # apply cycle
            c = np.array(list(M[:,j]))
            M[:,j] = M[:,i]
            M[:,i] = c
            # M[:,i], M[:,j] = M[:,j], M[:,i]
    return M

In [34]:
def test_my_perm(state, perm):
    """
    perform a permutation `perm`, given as a (list) of (tuple)s of (int)s, on a python (list) `state` using pure python loops and list operations.
    `perm` represents the cyclic notation of a permutation, each tuple is one cycle.
    """
    state = perform_action(state, perm)

def test_sp_perm(state, perm):
    """
    perform a sympy (Permutation) `perm` on a python (list) `state`
    """
    state = perm(state)

def test_mat_perm(state, perm_matrix):
    """
    perform a permutation given by a permutation matrix (np.array) of (int) on `state`, also given as a (np.array) of (int)
    """
    state = perm_matrix@state

def test_np_perm(state, perm):
    """
    perform a permutation `perm`, given as a (list) of (np.array)s of (int)s, on a (np.array) `state` using numpy array operations.
    """
    state = np_perm(state, perm)

def test_non_cycle_np_perm(state, mapping):
    """
    """
    state = state[mapping]

def test_non_cycle_np_perm_no_id(state, non_id_mapping):
    """
    """
    state = state[non_id_mapping]

In [35]:
def time_py_sp_np(perm, n=25):
    state = list(range(n))
    state_sp = list(range(n))
    state_np, perm_np = list_to_arrays(state, perm)
    perm_sp = Permutation(perm, size=n)
    perm_mat = get_perm_matrix(perm, n)
    perm_scpy = scpy.csr_matrix(perm_mat)
    np_mapping = non_cycle_np_perm(state_np, perm_np)
    non_id_mapping = non_id_cycle_np_perm(state_np, perm_np)
    
    %timeit test_my_perm(state, perm)
    %timeit test_sp_perm(state_sp, perm_sp)
    # %timeit test_mat_perm(state_np, perm_np)
    # %timeit test_mat_perm(state_np, perm_scpy)
    %timeit test_np_perm(state_np, perm_np)
    %timeit test_non_cycle_np_perm(state_np, np_mapping)
    %timeit test_non_cycle_np_perm_no_id(state_np, non_id_mapping)

In [6]:
Permutation??

In [44]:
perm1 = [[1,2,3], [4,5,0]]
perm2 = [[1,2], [3,4,5,0], [6,7,8,9,10,11], [12,13], [14,15,16], [17,18,19,20,21,22,23,24]]
perm3 = [[0,1,2], [4,6,7], [8,9,11,12,13,14], [16,17,19,20,21,22,23], [24,25,26,27,28,30], [31,32,34,40,41,42], [43,44,46], [48,49,50,52,54,55], [56,57,58,59,60,62,64,65,67,69,70,72,73,74,75], [78,79,80,82,83,84,85,86], [87,89,90], [91,92,93,94,95,96,97,98,99], [299,300]]

perm_3x3 = [[6, 33, 24, 15], [7, 34, 25, 16], [8, 35, 26, 17], [36, 42, 44, 38], [37, 39, 43, 41]]
perm_4x4 = [[12, 60, 44, 28], [13, 61, 45, 29], [14, 62, 46, 30], [15, 63, 47, 31], [64, 76, 79, 67], [65, 72, 78, 71], [66, 68, 77, 75], [69, 73, 74, 70]]

In [45]:
# time_py_sp_np(perm1, n=6)
# time_py_sp_np(perm2, n=100)
# time_py_sp_np(perm3, n=305)
# time_py_sp_np(perm_3x3, n=54)
time_py_sp_np(perm_4x4, n=144)

1.53 µs ± 22.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
3.74 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
19.4 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.16 µs ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
950 ns ± 8.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [71]:
perm = perm1
n = 6

state = list(range(n))
state_sp = list(range(n))
state_np = np.array(state, dtype=np.uint8)
perm_sp = Permutation(perm)
perm_mat = get_perm_matrix(perm, n)
print(perform_action(state, perm))
print(perm_sp(state_sp))
print(perm_mat@state_np)

[5, 3, 1, 2, 0, 4]
[4, 2, 3, 1, 5, 0]
[4. 2. 3. 1. 5. 0.]
